In [1]:
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np

#Init main values
symbols = "0123456789" # All symbols captcha can contain
print(symbols)
num_symbols = len(symbols)
print(num_symbols)
img_shape = (50, 182, 1)

Using TensorFlow backend.


0123456789
10


In [13]:
# Define a function that creates a net
def create_net():
    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 6 branches from it. Each branch will predict one letter
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(6):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    
    # Compile model and return it
    model = Model(img, outs)
    model.compile('rmsprop', loss=['categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy',
                                   'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'])
    return model

In [18]:
# First we need to preprocess the data
def preprocess_data():
    n_samples = len(os.listdir(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training\captcha'))
    X = np.zeros((n_samples, 50, 182, 1))
    y = np.zeros((6, n_samples, num_symbols))

    for i, pic in enumerate(os.listdir(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training\captcha')):
        # Read image as grayscale
        img = cv2.imread(os.path.join(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training\captcha', pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 7:
            # Scale and reshape image
            img = img / 255.
            img = np.reshape(img, (50, 182, 1))
            
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((6, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

X, y = preprocess_data()
X_train, y_train = X[:1000], y[:, :1000]
X_test, y_test = X[1000:], y[:, 1000:]

In [19]:
# Create net and fit
net = create_net()
history = net.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5]], batch_size=32, epochs=30, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 27s 34ms/step - loss: 16.5352 - dense_26_loss: 2.3936 - dense_28_loss: 2.8411 - dense_30_loss: 2.9827 - dense_32_loss: 2.7364 - dense_34_loss: 2.7611 - dense_36_loss: 2.8202 - val_loss: 20.2893 - val_dense_26_loss: 8.7036 - val_dense_28_loss: 2.3288 - val_dense_30_loss: 2.3063 - val_dense_32_loss: 2.3049 - val_dense_34_loss: 2.2962 - val_dense_36_loss: 2.3494
Epoch 2/30
800/800 [==============================] - 23s 29ms/step - loss: 14.0852 - dense_26_loss: 2.1321 - dense_28_loss: 2.3759 - dense_30_loss: 2.4031 - dense_32_loss: 2.4322 - dense_34_loss: 2.4396 - dense_36_loss: 2.3022 - val_loss: 18.4424 - val_dense_26_loss: 7.0127 - val_dense_28_loss: 2.2981 - val_dense_30_loss: 2.2998 - val_dense_32_loss: 2.2727 - val_dense_34_loss: 2.2630 - val_dense_36_loss: 2.2961
Epoch 3/30
800/800 [==============================] - 23s 29ms/step - loss: 13.7955 - dense_26_loss: 2.0492 - dense_28_los

Epoch 21/30
800/800 [==============================] - 22s 28ms/step - loss: 5.9243 - dense_26_loss: 0.3800 - dense_28_loss: 1.1582 - dense_30_loss: 1.1344 - dense_32_loss: 0.9801 - dense_34_loss: 0.9298 - dense_36_loss: 1.3419 - val_loss: 15.7003 - val_dense_26_loss: 11.9539 - val_dense_28_loss: 0.9536 - val_dense_30_loss: 0.7056 - val_dense_32_loss: 0.5959 - val_dense_34_loss: 0.5621 - val_dense_36_loss: 0.9293
Epoch 22/30
800/800 [==============================] - 23s 28ms/step - loss: 5.2097 - dense_26_loss: 0.3619 - dense_28_loss: 1.0718 - dense_30_loss: 1.0421 - dense_32_loss: 0.8387 - dense_34_loss: 0.7256 - dense_36_loss: 1.1696 - val_loss: 14.4667 - val_dense_26_loss: 11.0746 - val_dense_28_loss: 0.8005 - val_dense_30_loss: 0.5655 - val_dense_32_loss: 0.5643 - val_dense_34_loss: 0.5371 - val_dense_36_loss: 0.9247
Epoch 23/30
800/800 [==============================] - 22s 27ms/step - loss: 4.9167 - dense_26_loss: 0.3399 - dense_28_loss: 0.9372 - dense_30_loss: 0.9375 - dense_32

In [20]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) / 255.
    res = np.array(net.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (6, 10))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt, sum(probs) / 6

In [21]:
# Check model on some samples
net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4], y_test[5]])

print(predict(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training/captcha/108207.png'))
print(predict(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training/captcha/111183.png'))
print(predict(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training/captcha/111281.png'))

('108207', 0.5514591832955679)
('111183', 0.7993138382832209)
('111281', 0.6913224905729294)


In [22]:
#testing different captchas
print(predict(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training/01.png'))
print(predict(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training/02.png'))
print(predict(r'C:\Users\MudraCircle\.conda\pkgs\04_CAPTCHA model training/03.png'))

('103662', 0.6537480105956396)
('104639', 0.599955086906751)
('105773', 0.757027750213941)
